In [4]:
import json
import pandas as pd

# Pad naar de bestanden
food_file_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\__MACOSX\foodb_2020_04_07_json\Food.json"
content_file_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\__MACOSX\foodb_2020_04_07_json\Content.json"

# Food ID instellen waarop gefilterd moet worden
selected_food_id = 506  # Vervang dit nummer met het gewenste food_id

In [6]:
# Animal foods filteren om id's te bekomen
filtered_food_data = []
with open(food_file_path, "r", encoding="utf-8") as food_file:
    for line in food_file:
        try:
            item = json.loads(line.strip())
            if item.get("food_group") == "Animal foods":
                filtered_food_data.append({"id": item.get("id"), "name": item.get("name")})
        except json.JSONDecodeError as e:
            print(f"Fout bij het verwerken van een regel in food.json: {e}")

food_df = pd.DataFrame(filtered_food_data)

food_output_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\id_name_foodb.csv"
food_df.to_csv(food_output_path, index=False)
print(f"\nFood.json DataFrame opgeslagen als: {food_output_path}")

# Eerste paar regels van de DataFrame weergeven
print("Food.json eerste paar regels:")
print(food_df.head())


Food.json DataFrame opgeslagen als: C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\id_name_foodb.csv
Food.json eerste paar regels:
    id       name
0  303      Bison
1  310  Wild boar
2  316    Buffalo
3  334    Chicken
4  353  Mule deer


In [7]:
# content.json filteren op basis van food_id's & relevante kolommen selecteren
content_data = []
with open(content_file_path, "r", encoding="utf-8") as content_file:
    for line in content_file:
        try:
            item = json.loads(line.strip())
            if item.get("food_id") == selected_food_id:  # Filter op de geselecteerde food_id
                # Alleen de gewenste kolommen opnemen
                filtered_item = {
                    "id": item.get("id"),
                    "food_id": item.get("food_id"),
                    "orig_food_id": item.get("orig_food_id"),
                    "orig_food_common_name": item.get("orig_food_common_name"),
                    "source_id": item.get("source_id"),
                    "orig_source_id": item.get("orig_source_id"),
                    "orig_source_name": item.get("orig_source_name"),
                    "source_type": item.get("source_type"),
                    "orig_content": item.get("orig_content"),
                    "orig_unit": item.get("orig_unit"),
                    "standard_content": item.get("standard_content"),
                }
                content_data.append(filtered_item)
        except json.JSONDecodeError as e:
            print(f"Fout bij het verwerken van een regel in content.json: {e}")

# Data omzetten naar een Pandas DataFrame
content_df = pd.DataFrame(content_data)
print("content_data eerste paar regels:")
print(content_df.head())

content_data eerste paar regels:
     id  food_id orig_food_id              orig_food_common_name  source_id  \
0  4690      506         0551      Beef brisket, soaked in brine          2   
1  4691      506         0551      Beef brisket, soaked in brine          2   
2  4692      506         0436  Beef, brisket, anterior part, raw          2   
3  4693      506         0436  Beef, brisket, anterior part, raw          2   
4  4694      506         0437    Beef, brisket, middle part, raw          2   

  orig_source_id  orig_source_name source_type orig_content orig_unit  \
0           0001    Protein, total    Nutrient      21000.0  mg/100 g   
1           0002  Protein, total-N    Nutrient       3400.0  mg/100 g   
2           0001    Protein, total    Nutrient      17600.0  mg/100 g   
3           0002  Protein, total-N    Nutrient       2800.0  mg/100 g   
4           0001    Protein, total    Nutrient      15900.0  mg/100 g   

  standard_content  
0          21000.0  
1          

id zal per definitie altijd een waarde hebben, daarnaast wordt er gefilterd op food_id waardoor dit ook altijd een waarde zal hebben. orig_food_id is de id van orig_food_common_name, nulwaarden hiervoor zijn dus niet problematisch zolang orig_food_common_name een waarde heeft. Deze worden gebruikt om binnenin de food_id een verdere distinctie te maken tussen verschillende onderdelen van het geheel, bv food_id 506 = beef & binnenin beef zijn er verschillende onderdelen zoals brisket, sirloin, minced meat, fat etc. orig_source_name & orig_source_id worden dan gebruikt om de verschillende componenten te kenmerken zoals glutamic acid, glucose etc. Source_type wordt gebruikt om een distinctie te maken tussen nutrients (FAT, PROTEIN, ...) & compounds (Glutamic acid, glucose, ...). Orig_content, orig_unit & standard_content worden gebruikt om de waarde van elke nutrient/compound te bewaren en quantificeren. Tenslotte blijft source_id over wat nog steeds een compleet mysterie voor me blijft, de naam lijkt belangrijk, maar ik kan geen enkele connectie vinden met een andere parameter en ik heb nog geen antwoord ontvangen van de eigenaars van de database.

In [18]:
# Bereken het aantal unieke waarden per kolom
unique_values_per_column = content_df.apply(lambda col: col.nunique())

# Toon de resultaten
print("Aantal unieke waarden per kolom:")
print(unique_values_per_column)



Aantal unieke waarden per kolom:
id                       138620
food_id                       1
orig_food_id                908
orig_food_common_name       908
source_id                 49997
orig_source_id              241
orig_source_name            205
source_type                   2
orig_content              10720
orig_unit                     6
standard_content          10154
dtype: int64


In [13]:
# Controle A: Welke `orig_source_id`'s horen bij een `orig_source_name`?
source_name_to_ids = content_df.groupby("orig_source_name")["orig_source_id"].unique()
multiple_ids_per_name = source_name_to_ids[source_name_to_ids.apply(len) > 1]

total_unique_source_names = content_df["orig_source_name"].nunique()
print(f"\nTotaal aantal unieke orig_source_name's: {total_unique_source_names}")
print(f"Aantal orig_source_name met meerdere orig_source_id's: {len(multiple_ids_per_name)}")

if not multiple_ids_per_name.empty:
    print("\nVoorbeelden van orig_source_name met bijbehorende orig_source_id's (eerste 5):")
    for name, ids in multiple_ids_per_name.head().items():
        print(f"{name}: {list(ids)}")

# Bereken het aantal extra gebruikte ID's
total_ids_used = sum(source_name_to_ids.apply(len))  # Totaal aantal ID's gekoppeld aan namen
extra_ids = total_ids_used - total_unique_source_names  # Extra ID's gebruikt
print(f"\nTotaal aantal extra gebruikte orig_source_id's: {extra_ids}")

# Controle B: Welke `orig_source_name`'s horen bij een `orig_source_id`?
source_id_to_names = content_df.groupby("orig_source_id")["orig_source_name"].unique()
multiple_names_per_id = source_id_to_names[source_id_to_names.apply(len) > 1]

total_unique_source_ids = content_df["orig_source_id"].nunique()
print(f"\nTotaal aantal unieke orig_source_id's: {total_unique_source_ids}")
print(f"Aantal orig_source_id met meerdere orig_source_name's: {len(multiple_names_per_id)}")

if not multiple_names_per_id.empty:
    print("\nVoorbeelden van orig_source_id met bijbehorende orig_source_name's (eerste 5):")
    for source_id, names in multiple_names_per_id.head().items():
        print(f"{source_id}: {list(names)}")



Totaal aantal unieke orig_source_name's: 205
Aantal orig_source_name met meerdere orig_source_id's: 34

Voorbeelden van orig_source_name met bijbehorende orig_source_id's (eerste 5):
Alanine: ['0248', '513']
Arginine: ['0246', '511']
Ash: ['0012', '207']
Aspartic acid: ['0249', '514']
Carotene, beta: ['0016', '321']

Totaal aantal extra gebruikte orig_source_id's: 36

Totaal aantal unieke orig_source_id's: 241
Aantal orig_source_id met meerdere orig_source_name's: 0


Sommige orig_source_names zijn dus gekoppeld aan meerdere ID's, maar niet vice versa, dus dit vormt geen probleem. !!!Inspecteren per source_name waarom dit zo is, en indien dit gewoon een foutje is kunnen de ID's gecollapsed worden naar 1 waarde, of gewoon zo gelaten worden en de informatie van meerdere ID's onthouden voor latere verdere analyses/koppelingen te maken met de content.json dataset. KIJKEN OF DIT GEBRUIK AFHANKELIJK IS VAN EEN ANDERE PARAMETER, wss source_type of orig_unit aangezien dit de enige kolommen zijn met slechts een aantal verschillende soort waardes en aangezien er voor 32 names 2 ID's gebruikt worden en 2 names 3 ID's wijst dit waarschijnlijk op een heel eenvoudige relatie (indien die er is)!!!

In [9]:
# Controle of orig_content gelijk is aan standard_content
equal_orig_standard = (content_df["orig_content"] == content_df["standard_content"])

# Percentage van gelijkheid
percentage_equal = equal_orig_standard.mean() * 100
print(f"\nPercentage waarin orig_content gelijk is aan standard_content: {percentage_equal:.2f}%")

# Rijen waar de waarden verschillen (indien nodig)
different_values = content_df[~equal_orig_standard]
print(f"\nAantal rijen waarin orig_content niet gelijk is aan standard_content: {len(different_values)}")
if not different_values.empty:
    print("\nVoorbeelden van verschillende waarden:")
    print(different_values[["id","orig_content", "standard_content"]].head())


Percentage waarin orig_content gelijk is aan standard_content: 62.76%

Aantal rijen waarin orig_content niet gelijk is aan standard_content: 51618

Voorbeelden van verschillende waarden:
           id   orig_content standard_content
13771  118472  280.831739962           1175.0
13772  118473  213.432122371            893.0
13773  118474   323.61376673           1354.0
13774  118475  315.009560229           1318.0
13775  118476  196.940726577            824.0


!!!controleer of dit verschil in orig_content tov standard content afhankelijk is van de source type of andere parameter, idem zoals hierboven!!!

In [8]:
missing_percentage = (content_df.isnull().mean() * 100)
print("\nPercentage ontbrekende waarden per kolom:")
print(missing_percentage)


Percentage ontbrekende waarden per kolom:
id                        0.000000
food_id                   0.000000
orig_food_id             36.174434
orig_food_common_name     0.200548
source_id                 0.000000
orig_source_id           40.297937
orig_source_name         40.297937
source_type               0.000000
orig_content             35.978935
orig_unit                35.978935
standard_content         35.978935
dtype: float64


In [27]:
# Sinds ontbrekende waardes in een van de content kolommen geen waardevolle datapunten zijn worden deze eruit gefilterd.
# Idem voor source_id & source_name kolom. Veranderen indien source_id toch een betekenis heeft in deze context!

# Controle 1: Correlatie tussen content kolommen
content_missing = content_df["orig_content"].isnull()
unit_missing = content_df["orig_unit"].isnull()
standard_missing = content_df["standard_content"].isnull()

# Controle of ontbrekende waarden in één van de content-kolommen altijd ontbreken in de andere twee
content_correlation = (content_missing == unit_missing) & (unit_missing == standard_missing)

if content_correlation.all():
    print("✅ Een nulwaarde in één van de content-kolommen (orig_content, orig_unit, standard_content) impliceert een nulwaarde in de andere twee.")
else:
    print("❌ Inconsistenties gevonden in de content-kolommen. Controleer de data.")

# Controle 2: Correlatie tussen source kolommen
source_name_missing = content_df["orig_source_name"].isnull()
source_id_missing = content_df["orig_source_id"].isnull()

# Controle of een nulwaarde in orig_source_name altijd gepaard gaat met een nulwaarde in orig_source_id
source_correlation = source_name_missing == source_id_missing

if source_correlation.all():
    print("✅ Een nulwaarde in orig_source_name impliceert altijd een nulwaarde in orig_source_id (en vice versa).")
else:
    print("❌ Inconsistenties gevonden in de source-kolommen. Controleer de data.")

# Aantal rijen vóór filtering
original_row_count = len(content_df)

# Verwijder rijen waar een van de kolommen nulwaarden bevat
columns_to_check = ["orig_content", "orig_unit", "standard_content", "orig_source_name", "orig_source_id"]
filtered_df = content_df.dropna(subset=columns_to_check, how="any")

# Aantal rijen na filtering
filtered_row_count = len(filtered_df)

# Bereken het percentage ontbrekende waarden opnieuw per kolom
missing_percentage = filtered_df.isnull().mean() * 100

# Toon het resultaat
print(f"\nAantal rijen oorspronkelijk: {original_row_count}")
print(f"Aantal rijen na filtering: {filtered_row_count}")
print(f"Aantal verwijderde rijen: {original_row_count - filtered_row_count}")
print("\nPercentage ontbrekende waarden per kolom na filtering:")
print(missing_percentage)


✅ Een nulwaarde in één van de content-kolommen (orig_content, orig_unit, standard_content) impliceert een nulwaarde in de andere twee.
✅ Een nulwaarde in orig_source_name impliceert altijd een nulwaarde in orig_source_id (en vice versa).

Aantal rijen oorspronkelijk: 138620
Aantal rijen na filtering: 82758
Aantal verwijderde rijen: 55862

Percentage ontbrekende waarden per kolom na filtering:
id                       0.0
food_id                  0.0
orig_food_id             0.0
orig_food_common_name    0.0
source_id                0.0
orig_source_id           0.0
orig_source_name         0.0
source_type              0.0
orig_content             0.0
orig_unit                0.0
standard_content         0.0
dtype: float64


In [28]:
# Bereken het aantal unieke waarden per kolom
unique_values_per_column2 = filtered_df.apply(lambda col: col.nunique())

# Toon de resultaten
print("Aantal unieke waarden per kolom:")
print(unique_values_per_column2)


Aantal unieke waarden per kolom:
id                       82758
food_id                      1
orig_food_id               907
orig_food_common_name      907
source_id                  139
orig_source_id             240
orig_source_name           204
source_type                  2
orig_content             10622
orig_unit                    6
standard_content         10056
dtype: int64
